In [ ]:
%pip install mysql-connector-python openpyxl pandas

In [27]:
import pandas as pd
from datetime import datetime
from tkinter import Tk, filedialog

# Tạo hộp thoại chọn file
root = Tk()
root.withdraw()  # Ẩn cửa sổ chính của tkinter
root.attributes('-topmost', True)  # Đưa hộp thoại lên trên cùng

# Mở hộp thoại chọn file Excel
file_path = filedialog.askopenfilename(
    title="Chọn file Excel",
    filetypes=[("Excel files", "*.xlsx *.xls"), ("All files", "*.*")]
)

if file_path:
    print(f"Đã chọn file: {file_path}")
    df = pd.read_excel(file_path, sheet_name=3, dtype=str)
    df_copy = df.copy()
    print(f"Đã đọc file, có {len(df)} dòng")
else:
    print("Không có file nào được chọn!")
    df_copy = None

Đã chọn file: C:/Users/doman/Downloads/Yeu cau moi.xlsx
Đã đọc file, có 153 dòng


In [28]:
df_copy = df_copy.loc[:, ~df_copy.columns.str.contains("^Unnamed")]

# 3. Danh sách cột theo thứ tự
columns = df_copy.columns.tolist()

if "id" in columns:
    columns.remove("id")

# 4. Sinh câu INSERT
insert_statements = []
for _, row in df_copy.iterrows():
    values = []
    for col in columns:
        val = row[col]

        if col == "created_at" or col == "updated_at":
            values.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        elif pd.isna(val):
            values.append("NULL")
        elif isinstance(val, bool):
            values.append("TRUE" if val else "FALSE")
        elif isinstance(val, (int, float)):
            values.append(str(val))
        else:
            # string phải có dấu nháy đơn
            safe_val = str(val).replace("'", "''")  # escape dấu nháy đơn trong SQL
            values.append("N'" + safe_val + "'")

    insert_statements.append(
        f"INSERT INTO Sector_Route_DOM_Ref ({', '.join(columns)})"
        + f"VALUES ({', '.join(values)});"
    )

# 5. In thử vài dòng
for stmt in insert_statements[:3]:
    print(stmt)

INSERT INTO Sector_Route_DOM_Ref (Sector, Area lv1, DOM/INT, created_at, updated_at)VALUES (N'BMV-PQC', N'Du lịch', N'DOM', 2025-11-03 18:27:40, 2025-11-03 18:27:40);
INSERT INTO Sector_Route_DOM_Ref (Sector, Area lv1, DOM/INT, created_at, updated_at)VALUES (N'PQC-BMV', N'Du lịch', N'DOM', 2025-11-03 18:27:40, 2025-11-03 18:27:40);
INSERT INTO Sector_Route_DOM_Ref (Sector, Area lv1, DOM/INT, created_at, updated_at)VALUES (N'PQC-CXR', N'Du lịch', N'DOM', 2025-11-03 18:27:40, 2025-11-03 18:27:40);


In [29]:
# 1. Tạo phần đầu câu INSERT
insert_sql = f"INSERT INTO Sector_Route_DOM_Ref ({', '.join(columns)})\nVALUES\n"

# 2. Tạo danh sách values
values_list = []
for _, row in df_copy.iterrows():
    row_values = []
    for col in columns:
        val = row[col]

        if col == "created_at" or col == "updated_at":
            row_values.append("SYSDATETIME()")
        elif pd.isna(val):
            row_values.append("NULL")
        elif isinstance(val, bool):
            row_values.append("TRUE" if val else "FALSE")
        elif isinstance(val, (int, float)):
            row_values.append(str(val))
        else:
            safe_val = str(val).replace("'", "''")
            row_values.append(f"N'{safe_val}'")

    values_list.append(f"({', '.join(row_values)})")

# 3. Ghép toàn bộ VALUES vào 1 câu SQL
limit = 1000
skip = 0

while skip < len(values_list):
    final_sql = insert_sql + ",\n".join(values_list[skip:skip + limit]) + ";"
    print(final_sql)  # in thử 1000 ký tự đầu
    skip += limit

# final_sql = insert_sql + ",\n".join(values_list) + ";"

# print(final_sql)  # in thử 1000 ký tự đầu

INSERT INTO Sector_Route_DOM_Ref (Sector, Area lv1, DOM/INT, created_at, updated_at)
VALUES
(N'BMV-PQC', N'Du lịch', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'PQC-BMV', N'Du lịch', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'PQC-CXR', N'Du lịch', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'CXR-PQC', N'Du lịch', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'VCA-CXR', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'CXR-VCA', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'DAD-BMV', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'BMV-DAD', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'DAD-CXR', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'CXR-DAD', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'DAD-DLI', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'DLI-DAD', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'HPH-DAD', N'Địa phương_2', N'DOM', SYSDATETIME(), SYSDATETIME()),
(N'DAD-HPH', N'Địa phương_2', N